In [ ]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

from matplotlib import pyplot
from sklearn.tree import plot_tree
from matplotlib.pyplot import figure

from sklearn.model_selection import train_test_split, GridSearchCV
random_state = 1
train_size = 0.75
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import f_regression

from sklearn.tree import DecisionTreeClassifier  


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error, r2_score
import scipy.stats
# Computation of F-statistic and p-value for the regression
# http://facweb.cs.depaul.edu/sjost/csc423/documents/f-test-reg.htm
def f_test(y_true, y_pred, n_var, n_obs, sn=.95):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    n = n_obs
    p = n_var+1 # number of regression parameters (coefficients + intercept)
    y_true_m = np.mean(y_true)
    SSM = np.sum((y_pred-y_true_m)**2)
    SST = np.sum((y_true-y_true_m)**2)
    SSE = np.sum((y_true-y_pred)**2)
    DFT = n - 1
    DFM = p - 1 # degrees of freedom for model - numerator
    DFE = n - p # degrees of freedom for error - denominator
    DFT = n - 1
    MSM = SSM / DFM
    MSE = SSE / DFE 
    MST = SST / DFT
    F = MSM / MSE
    p = 1-scipy.stats.f.cdf(F, DFM, DFE) #find p-value of F test statistic 
    return F, p
    
def print_eval(X, y, model):
    pred = model.predict(X)
    F, p = f_test(y, pred, X.shape[1], X.shape[0])
    print(" Mean squared error: \t{:.5}".format(mean_squared_error(y,pred)))
    print(" r2 score: \t\t{:.5}".format(r2_score(y,pred)))
    print(" f-statistic: \t\t{:.5}".format(F))
    print(" p-value: \t\t{:.5}".format(p))
    return mean_squared_error(pred, y), r2_score(pred, y), F, p

In [ ]:
# Read data from file (or url) and save the dataframe
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
df = pd.read_csv(url, sep = ';') 
# if the names of the columns are not present, insert them using `names = []`
# if the file is an excel use df = pd.read_excel(data_fn)
print(f"Shape of the input data {df.shape}")

target = 'quality'

# storing in X the content of the dataframe excluding the target column
X = df.drop(target, axis=1)
# storing in y the labels
y = df[target]
print(f"Shape of X: {X.shape}\nShape of y: {y.shape}")

# dividing the dataset in train and test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=random_state, train_size = train_size)
print("There are {} samples in the training dataset".format(Xtrain.shape[0]))
print("There are {} samples in the testing dataset".format(Xtest.shape[0]))
print("Each sample has {} features".format(Xtrain.shape[1]))

In [ ]:
# Show the p-values of the target with respect to the variables
_, p_values = f_regression(X,y)
p_values_show = pd.DataFrame({'Variable': X.columns, 'p-value': p_values})
p_values_show

# Linear Regression

In [ ]:
pred_var = 'adults_n'
# use those sets to performe univariate linear regression
X_train_r = Xtrain[pred_var].values.reshape(-1,1) # transform a series into a one-column array
X_test_r = Xtest[pred_var].values.reshape(-1,1)

In [ ]:
# Create linear regressor object
linear_multi = LinearRegression()
# Train the model using the training set
linear_multi.fit(Xtrain, ytrain)

# Make predictions using the test set
y_train_pred_multi = linear_multi.predict(Xtrain)
y_test_pred_multi = linear_multi.predict(Xtest)

# Show the coefficients of the predicting variables
pd.DataFrame({'Variable': X.columns, 'Coefficient': linear_multi.coef_})

In [ ]:
# computing the statistical significance
_, p_values = f_regression(Xtrain,y_train_pred_multi)
p_values_show = pd.DataFrame({'Variable': X.columns, 'p-value': p_values})
p_values_show

In [ ]:
# compute the quality measures

#perform F-test
f_statistic_multi, p_value_multi = f_test(ytrain, y_train_pred_multi
                                        , Xtrain.shape[1], Xtrain.shape[0])
                                        
# The mean squared error
rmse_multi = mean_squared_error(ytest, y_test_pred_multi, squared=False)
# print("The MSE for the multivariate linear regression of '{target}' is: {rmse_dt:8.2f}")

# Coefficient of determination=1 is perfect prediction
r2_multi = r2_score(ytest, y_test_pred_multi)
# print("The 'R square' for the multivariate linear regression of '{target}' is: {r2_dt:8.2f}")

pd.DataFrame({'Univariate Linear - Value' : [rmse_multi
                        , r2_multi
                        , f_statistic_multi
                        , p_value_multi]}
            , index = ['rmse'
                     , 'r2'
                     , 'f-statistic'
                     , 'p-value']).style.format(precision=4)

## Decision Tree Multivariate Regression

In [ ]:
# create the Decision Tree regressor object
dt = DecisionTreeRegressor(random_state=random_state)
# Train the model using the training set
dt.fit(Xtrain, ytrain);
max_max_depth = dt.tree_.max_depth
print("The maximum depth of the full Decision Tree Regressor is {}".format(max_max_depth))

In [ ]:
# creating the gris search cross validation object 
# and finding the best depth of the tree
param_grid = {'max_depth': list(range(1,max_max_depth))}
dt_gscv = GridSearchCV(estimator=DecisionTreeRegressor(random_state=random_state)
                    , param_grid=param_grid
                    , scoring='neg_mean_squared_error' # select model with minimum mse
                    )
dt_gscv.fit(Xtrain,ytrain);
dt_best = dt_gscv.best_estimator_ # the GridSearchCV returns the best estimator
best_max_depth = dt_best.tree_.max_depth
print(f"The optimal maximum depth for the decision tree is {best_max_depth}")

In [ ]:
# Make predictions using the test set
y_test_pred_dt = dt_best.predict(Xtest)
rmse_dt = mean_squared_error(ytest, y_test_pred_dt, squared=False)
print(f"Decision Tree Regression - RMSE = {rmse_dt:.2f}")

In [ ]:
# show the tree
figure(figsize = (20,15))
plot_tree(dt_best
          , filled = True # fills nodes with colors related to classes
                          # darker color means higher purity
          , feature_names = X.columns
          , fontsize=18
        #   , class_names = df[target].unique()
         );

## Random Forest Multivariate Regression

In [ ]:
# Create Random Forest regression object
rf = RandomForestRegressor(random_state=random_state)
# for simplicity, we use as a maximum maximum depth of the tree the value found in
# the unconstrained decision tree fitting
param_grid_rf = {'max_depth': list(range(1,max_max_depth))
}
# create the grid search with cross validation
rf_gscv = GridSearchCV(rf, param_grid=param_grid_rf
                        , scoring='neg_mean_squared_error') # look for minimum mean square error

# Train the model using the training set
rf_gscv.fit(Xtrain, ytrain)

# the grid search returns the best estimator
rf = rf_gscv.best_estimator_

print("The optimal maximum depth for the trees in the random forest is {}".format(rf.max_depth))

y_test_pred_rf = rf.predict(Xtest)
rmse_rf = mean_squared_error(ytest, y_test_pred_rf, squared=False)
print("Random Forest Regression - RMSE = {rmse_rf:.2f}")

# Polynomial regression

In [ ]:
# showing the distribution of the data
plt.scatter(X,y)
plt.xlabel("Temperature");
plt.ylabel("Demand");
plt.show()

# divide the data in train and test

In [ ]:
# first experiment with a  linear model
lmodel = LinearRegression()
lmodel.fit(X.temp.values.reshape(-1,1), y)
lin = print_eval(Xtest, ytest, lmodel)
# visualizing the prediction of the model
lpred = lmodel.predict(np.arange(min(X.temp), max(X.temp)).reshape(-1,1))
plt.plot(np.arange(min(X.temp), max(X.temp)),lpred, label = "lin", color = "red")
plt.legend()
plt.scatter(X,y)
plt.show();

In [ ]:
# second experiment with a polynomial regressor
polFea = PolynomialFeatures(2,include_bias=False)
X_poly = polFea.fit_transform(Xtrain.values)#.reshape(-1,1))
model = LinearRegression()
model.fit(X_poly, ytrain)
pol = print_eval(polFea.transform(Xtest), ytest, model)

# plotting the polynomial regressor
pred = model.predict(polFea.transform((np.arange(min(X.temp), max(X.temp))).reshape(-1,1)))
plt.plot(np.arange(min(X.temp), max(X.temp)),pred, label = "poly",color="red")
plt.legend()
plt.scatter(X,y)
plt.plot();

In [ ]:
# third expreriment
polFea = PolynomialFeatures(3,include_bias=False)
print("Polynomial degree = 3")
X_poly = polFea.fit_transform(Xtrain.values)#.reshape(-1,1))
model = LinearRegression()
model.fit(X_poly, ytrain)

pol3 = print_eval(polFea.transform(Xtest), ytest, model)

pred3 = model.predict(polFea.transform((np.arange(min(X.temp), max(X.temp))).reshape(-1,1)))
plt.plot(np.arange(min(X.temp), max(X.temp)),pred3, label = "poly",color="red")
plt.legend()
plt.scatter(X,y)
plt.plot();

In [ ]:
# comparing the performance of the models
performance = {"linear ": [*lin],
                "polynomial d = 2": [*pol],
                "polynomial d = 3": [*pol3] }
res = pd.DataFrame(performance, index = ['rmse'
                     , 'r2'
                     , 'f-statistic'
                     , 'p-value'])
res